# ASR evaluation
we perform subjective evaluation based on 
1. CER/WER by strong ASR
2. estimated SI-SNR
The dataset includes "clean"(AISHELL), "noisy"(wild V2S), "processed"(V2S_tmp)

In [1]:
import scipy.io.wavfile as wavfile
import librosa
import os
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import numpy as np


In [2]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference_16k_pipline = pipeline(
    task=Tasks.auto_speech_recognition,
    model='damo/speech_paraformer_asr_nat-aishell1-pytorch', device='gpu')

rec_result = inference_16k_pipline(audio_in='https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_audio/asr_example_zh.wav')
print(rec_result)

2023-09-14 09:48:20,744 - modelscope - INFO - PyTorch version 1.13.0 Found.
2023-09-14 09:48:20,745 - modelscope - INFO - Loading ast index from /home/lixing/.cache/modelscope/ast_indexer
2023-09-14 09:48:20,760 - modelscope - INFO - Loading done! Current index file version is 1.8.4, with md5 bc2412af0ceee67320f5157c651de533 and a total number of 902 components indexed
2023-09-14 09:48:22,583 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.8
2023-09-14 09:48:23,055 - modelscope - INFO - initiate model from /home/lixing/.cache/modelscope/hub/damo/speech_paraformer_asr_nat-aishell1-pytorch
2023-09-14 09:48:23,055 - modelscope - INFO - initiate model from location /home/lixing/.cache/modelscope/hub/damo/speech_paraformer_asr_nat-aishell1-pytorch.
2023-09-14 09:48:23,056 - modelscope - INFO - initialize model from /home/lixing/.cache/modelscope/hub/damo/speech_paraformer_asr_nat-aishell1-pytorch
2023-09-14 09:48:23,058 - modelscope - WARNING - No preproces

{'text': '欢迎大家来体验打磨愿推出的语音识别模型'}


In [6]:
'''
prepare processed data into _tmp folder
1. separate to 1-st channel (vibration) or 2-nd channel (microphone)
'''
new_rate = 16000
dataset = '../V2S/'
dataset_target = '../V2S_tmp/'
for speaker in os.listdir(dataset):
    if not os.path.exists(dataset_target + speaker):
        os.mkdir(dataset_target + speaker)
    for date in os.listdir(dataset + speaker):
        print(speaker, date)
        if not os.path.exists(dataset_target + speaker + "/" + date):
            os.mkdir(dataset_target + speaker + "/" + date)
        for wav in tqdm(os.listdir(dataset + speaker + "/" + date)):
            if wav[-3:] != "wav":
                shutil.copy(dataset + speaker + "/" + date + "/" + wav, dataset_target + speaker + "/" + date + "/" + wav)
                continue
            path = dataset + speaker + "/" + date + "/" + wav
            data, sample_rate = librosa.load(path, mono=False, sr=None)
            samples = round(len(data) * float(new_rate) / sample_rate)
            new_data = librosa.resample(data[1], orig_sr=sample_rate, target_sr=new_rate, scale=True)
            wavfile.write(dataset_target + speaker + "/" + date + "/" + wav, new_rate, new_data.T)

Bian_Chen 2023-09-07


100%|██████████| 75/75 [00:00<00:00, 82.64it/s]


Shaoyang_Yang 2023-09-10


100%|██████████| 107/107 [00:00<00:00, 2638.38it/s]


Haozheng_Hou 2023-09-11


...

100%|██████████| 83/83 [00:00<00:00, 2472.74it/s]


Lixing_He 2023-09-09


100%|██████████| 85/85 [00:00<00:00, 2548.56it/s]


Lixing_He 2023-09-05


100%|██████████| 135/135 [00:00<00:00, 2150.32it/s]


In [4]:
'''
evaluation 
python inference.py
'''
import jiwer
data = '../V2S_tmp/'
output = {}
for speaker in os.listdir(data):
    directory = os.path.join(data, speaker)
    for date in os.listdir(directory):
        hypotheses = []
        references = []
        directory_date = os.path.join(directory, date)
        labels = os.path.join(directory_date, 'labels.txt')
        labels = open(labels, 'r').readlines()
        for label in tqdm(labels):
            l = label.strip().split(' ')
            file = l[0]
            text = ''.join(l[1:])
            file = os.path.join(directory_date, file+'.wav')
            try: 
                rec_result = inference_16k_pipline(audio_in=file,)
                hypotheses.append(rec_result['text'])
                references.append(text)
            except:
                pass
        wer = jiwer.cer(hypotheses, references)
        output[speaker + '_' + date] = round(wer * 100, 2)
print(output)   

  0%|          | 0/75 [00:00<?, ?it/s]2023-09-14 12:14:00,623 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,703 - modelscope - INFO - Computing the result of ASR ...
2023-09-14 12:14:00,704 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,756 - modelscope - INFO - Computing the result of ASR ...
  3%|▎         | 2/75 [00:00<00:04, 14.79it/s]2023-09-14 12:14:00,757 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,811 - modelscope - INFO - Computing the result of ASR ...
2023-09-14 12:14:00,812 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,873 - modelscope - INFO - Computing the result of ASR ...
  5%|▌         | 4/75 [00:00<00:04, 16.05it/s]2023-09-14 12:14:00,874 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,935 - modelscope - INFO - Computing the result of ASR ...
2023-09-14 12:14:00,936 - modelscope - INFO - Decoding with wav files ...
2023-09-14 12:14:00,997 - modelscope

{'Bian_Chen_2023-09-07': 12.86, 'Shaoyang_Yang_2023-09-10': 22.58, 'Haozheng_Hou_2023-09-11': 31.21, 'Bowen_Zheng_2023-09-07': 7.74, 'Kaiwei_Liu_2023-09-11': 26.4, 'Youdong_Wang_2023-09-11': 19.14, 'Bufang_Yang_2023-09-09': 25.87, 'Bufang_Yang_2023-09-05': 17.31, 'Sitong_Cheng_2023-09-07': 4.57, 'Lixing_He_2023-09-10': 17.28, 'Lixing_He_2023-09-11': 42.13, 'Lixing_He_2023-09-09': 14.9, 'Lixing_He_2023-09-05': 7.29}


In [ ]:

folder = '../ABCS/'
split = 'dev'
data = os.path.join(folder, 'Audio', split)
label_folder = os.path.join(folder, 'script', split)
hypotheses = []
references = []
for labels in os.listdir(label_folder):
    speaker = labels.split('_')[0]
    labels = open(os.path.join(label_folder, labels), 'r').readlines()
    directory = os.path.join(data, speaker)
    for label in tqdm(labels):
        l = label.strip().split(' ')
        file = l[0]
        text = ''.join(l[1:])
        file = os.path.join(directory, file+'.wav')
        rec_result = inference_16k_pipline(audio_in=file,)
        print(rec_result['text'], text)
        hypotheses.append(rec_result['text'])
        references.append(text)
        break

import jiwer
wer = jiwer.cer(hypotheses, references)
print(f"CER: {wer * 100:.2f} %") 

In [14]:
'''
evaluation for estimated SISNR
python inference.py
'''
from speechbrain.pretrained.interfaces import SNREstimator as snrest
import torch
import torchaudio

# 3- Estimate the performance
snr_est_model = snrest.from_hparams(source="speechbrain/REAL-M-sisnr-estimator",savedir='pretrained_models/REAL-M-sisnr-estimator')
# snrhat = snr_est_model.estimate_batch(mix, est_sources)
# print(snrhat) # Estimates are in dB / 10 (in the range 0-1, e.g., 0 --> 0dB, 1 --> 10dB)

mix_data = '../V2S/'
data = '../V2S_tmp/'
speaker_whitelist = ['Bufang_Yang', 'Lixing_He']
date_whitelist = ['2023-09-05', '2023-09-07']
for speaker in os.listdir(data):
    if speaker not in speaker_whitelist:
        continue
    directory = os.path.join(data, speaker)
    for date in os.listdir(directory):
        if date not in date_whitelist:
            continue
        directory_date = os.path.join(directory, date)
        labels = os.path.join(directory_date, 'labels.txt')
        labels = open(labels, 'r').readlines()
        for label in tqdm(labels):
            l = label.strip().split(' ')
            file = l[0]
            text = ''.join(l[1:])
            file = os.path.join(directory_date, file+'.wav')
            est_source, fs = torchaudio.load(file)
            # mix_file = file.replace(data, mix_data)
            # mix, fs = torchaudio.load(mix_file)
            # mix = mix[1:, :]
            # est_source *= mix.max() / est_source.max()
            # est_noise = mix - est_source
            # est_sources = torch.stack([est_source, est_noise], dim=-1)

            est_sources = est_source[:, :, None].repeat(1, 1, 2)
            mix = est_source * 2
            # fig, axs = plt.subplots(2)
            # axs[0].plot(mix[0, :])
            # axs[1].plot(est_sources[0, :])
            # plt.show()
            snrhat = snr_est_model.estimate_batch(mix, est_sources)
            print(snrhat)
            break


  0%|          | 0/143 [00:00<?, ?it/s]


tensor([0.4867, 0.4867])


  0%|          | 0/134 [00:00<?, ?it/s]

tensor([1.1396, 1.1395])
